In [1]:
folder_path = "../data/jss/imrg/ds1/testbed_1_s"
best_path = "../data/jss/imrg/ds1/best/testbed1_small.csv"
output_file = "results/meal.csv"

In [28]:
import os, csv, time, math
import numpy as np
from mealpy import TS, SA, ACOR, GA, Problem, PermutationVar


class JSSP(Problem):
    def __init__(self, bounds, minmax, name="JSSP", processing_times=None, **kwargs):
        self.name = name
        self.processing_times = processing_times
        super().__init__(bounds, minmax, **kwargs)

    def obj_func(self, solution):
        solution = self.decode_solution(solution)["per"]
        num_jobs = len(solution)
        num_machines = len(self.processing_times[0])
        
        # Initialize completion times matrix
        completion_times = [[0] * num_machines for _ in range(num_jobs)]
        total_completion_time = 0

        for job_index in range(num_jobs):
            job = solution[job_index]

            for machine in range(num_machines):
                if job_index == 0:
                    if machine == 0:
                        completion_times[job_index][machine] = self.processing_times[job][machine]
                    else:
                        completion_times[job_index][machine] = completion_times[job_index][machine-1] + self.processing_times[job][machine]
                else:
                    if machine == 0:
                        completion_times[job_index][machine] = completion_times[job_index-1][machine] + self.processing_times[job][machine]
                    else:
                        completion_times[job_index][machine] = self.processing_times[job][machine] + max(completion_times[job_index][machine-1], completion_times[job_index-1][machine])

                if machine == num_machines - 1:
                    total_completion_time += completion_times[job_index][machine]

        return total_completion_time
    
    def amend_position(self, solution):
        unique_jobs = np.unique(solution)
        if len(unique_jobs) == len(solution):
            # The solution is already a valid permutation
            return solution
        else:
            # If it's not, we reconstruct a valid permutation
            # This scenario might be rare depending on how the solution is generated or modified
            n = len(solution)
            valid_permutation = np.random.permutation(n)
            return valid_permutation.tolist()

In [3]:
def read_file_input(filepath):
    with open(filepath) as f:
        lines = f.readlines()
        sizes = [int(nr) for nr in lines[0].split()]
        m, n = sizes[0], sizes[1]
        arr = np.zeros((n, m))

        for i in range(1, len(lines)):
            arr[i-1, :] = [int(nr) for nr in lines[i].split()]

    return arr

In [4]:
def read_all_files_in_folder(folder_path):
    # List to hold file contents
    file_contents = {}

    # Check if the provided path is a directory
    if not os.path.isdir(folder_path):
        raise ValueError(f"The provided path '{folder_path}' is not a valid directory.")

    # Iterate over all files in the directory
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        # Check if it's a file
        if os.path.isfile(file_path):
            try:
                # Open and read the file
                with open(file_path, 'r') as file:
                    content = file.read()
                    file_contents[filename] = content
            except Exception as e:
                print(f"Error reading file {filename}: {e}")
    
    return file_contents

In [153]:
def process_files(file_data):
    results = []
    total_time = 0
    for filename, content in file_data.items():
        data = read_file_input(f"{folder_path}/{filename}")
        problem_jssp = JSSP(bounds=PermutationVar(valid_set=[i for i in range(data.shape[0])], name="per"), processing_times=data, minmax="min", log_to="")
        # model = SA.OriginalSA(epoch=1400, temp_init=100, cooling_rate=0.995)
        # FIXME: seems like tabu search is bugged
        # model = TS.OriginalTS(epoch=1000, pop_size=50, tabu_size=5, neighbour_size=20, perturbation_scale=0.05)
        # model = GA.BaseGA(epoch=500, pop_size=50, pc=0.99, pm=0.75, selection="tournament", k_way=0.25, cross_over="one_point")
        model = ACOR.OriginalACOR(epoch=1400, pop_size=20, sample_count=20, intent_factor=0.1, zeta=0.1)

        start_time = time.time()
        model.solve(problem=problem_jssp)
        end_time = time.time()
        duration = end_time - start_time

        print(f"Processing file: {filename} took {duration:.4f}s")
        print(f"Schedule: {model.g_best.solution} Cost: {model.g_best.target.fitness}")

        results.append({"filename": filename, "cost": model.g_best.target.fitness, "time": duration})
        total_time += duration

    print(f"Total time spent solving: {total_time:.4f}s")
    return results

In [6]:
def write_results_to_csv(results, output_file):
    with open(output_file, 'w', newline='') as csvfile:
        fieldnames = ['filename', 'cost', 'time']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        for result in results:
            writer.writerow(result)

In [7]:
file_data = read_all_files_in_folder(folder_path)

In [154]:
results = process_files(file_data)

Processing file: t1s_0238.txt took 24.3377s
Schedule: [8 3 1 2 7 9 6 4 0 5] Cost: 12813.0
Processing file: t1s_0216.txt took 22.9181s
Schedule: [4 9 8 2 0 5 1 6 3 7] Cost: 13488.0
Processing file: t1s_0312.txt took 25.5773s
Schedule: [10  1  6  9  5  2  4 11  8  3  0  7] Cost: 11067.0
Processing file: t1s_0281.txt took 24.4812s
Schedule: [10  1  8  4  3  6  5  7  2 11  0  9] Cost: 6847.0
Processing file: t1s_0113.txt took 12.4145s
Schedule: [0 4 3 1 2] Cost: 6157.0
Processing file: t1s_0082.txt took 11.5849s
Schedule: [1 0 3 4 2] Cost: 3271.0
Processing file: t1s_0192.txt took 21.8755s
Schedule: [8 0 1 3 4 5 6 7 2 9] Cost: 8685.0
Processing file: t1s_0266.txt took 23.4254s
Schedule: [ 5  2  1  8 10  0 11  3  4  9  7  6] Cost: 4182.0
Processing file: t1s_0021.txt took 11.0044s
Schedule: [0 4 2 1 3] Cost: 947.0
Processing file: t1s_0018.txt took 11.1388s
Schedule: [2 0 1 3 4] Cost: 805.0
Processing file: t1s_0162.txt took 20.2246s
Schedule: [2 0 6 3 4 5 1 8 7 9] Cost: 5598.0
Processing f

In [155]:
results.sort(key=lambda x: x["filename"])
write_results_to_csv(results, output_file)

# Results

## SA

| temp_init | cooling_rate | epoch | Time |
|-----------|--------------|------------|------|
| 1000 | 0.9995 | 41500 | 4783s |
| 1000 | 0.9995 | 27500 | 3165s |
| 100 | 0.9995 | 13800 | 1519s |
| 100 | 0.999 | 6900 | 748s |
| 100 | 0.995 | 1400 | 132s |

## GA

| generations | time |
|-------------|------|
| 13800 | s |
| 6900 | s |
| 1400 | 5074s |
| 1000 | 3753s |
| 500 | 1851s |
| 200 | 975s |
| 100 | 614s |

## ACO

| generations | time |
|-------------|------|
| 13800 | s |
| 6900 | s |
| 1400 | 8231s |
| 1000 | 6499s |
| 500 | 3410s |
| 200 | 1534s |
| 100 | 675s |

# Tuning Params

In [30]:
from skopt.space import Real, Integer
from skopt import gp_minimize

In [50]:
def compute_overall_score_SA(params):
    # epoch, temp_init, cooling_rate = params
    cooling_rate = params[0]
    total_score = 0

    for filename, content in file_data.items():
        data = read_file_input(f"{folder_path}/{filename}")
        problem_jssp = JSSP(bounds=PermutationVar(valid_set=[i for i in range(data.shape[0])], name="per"), processing_times=data, minmax="min", log_to="")
        model = SA.OriginalSA(epoch=1400, temp_init=100, cooling_rate=cooling_rate)
        model.solve(problem=problem_jssp)

        total_score += model.g_best.target.fitness

    return total_score

In [51]:
# Define the search space for the parameters
space = [
    # Integer(1000, 15000, name='epoch'),
    # Real(50, 500, name='temp_init'),
    Real(0.95, 0.9995, name='cooling_rate')
]

# Perform Bayesian optimization
res = gp_minimize(compute_overall_score_SA, space, n_calls=100, random_state=0)

0.9793458086021384
2532622.0
0.9917911545547604
2520622.0
0.9924683080723266
2540505.0
0.9919389610698143
2531689.0
0.9808664029909057
2532680.0
0.9690268945109887
2532113.0
0.9647279630239514
2529200.0
0.9528072923772134
2543438.0
0.9634964865817156
2536788.0
0.9736444233074069
2529970.0
0.9913422301822873
2529876.0
0.9916723575001553
2523083.0
0.9916023553256337
2523612.0
0.9655785741827382
2527051.0
0.9662194927246258
2541983.0
0.9651952876975529
2543630.0
0.9915950782606657
2535132.0
0.9976949843633858
2528549.0
0.9995
2535930.0
0.9916588590781203
2525435.0
0.9916597300798639
2542739.0
0.9915303233059177
2525820.0
0.9916078074180464
2536860.0
0.99163650506425
2523959.0
0.9910935092836725
2525621.0
0.9890342847701454
2536327.0
0.9913848405122606
2540986.0
0.9916785882064448
2529502.0
0.991692308051166
2532066.0
0.9916906701943395
2531290.0
0.9930111381386757
2531255.0
0.9916905085409875
2534227.0
0.9916767661837214
2527157.0
0.9916994429888827
2533579.0
0.9916819949623503
2538466.0


In [53]:
# Output the best parameters found
print("Best parameters found:")
print(f"k: {res.x[0]}")

print("Best score achieved:")
print(res.fun)

Best parameters found:
k: 0.9994994149984398
Best score achieved:
2517532.0


In [93]:
def compute_overall_score_GA(params):
    pc, pm, k_way = params
    total_score = 0

    print(f"Params: {params}")

    for filename, content in file_data.items():
        data = read_file_input(f"{folder_path}/{filename}")
        problem_jssp = JSSP(bounds=PermutationVar(valid_set=[i for i in range(data.shape[0])], name="per"), processing_times=data, minmax="min", log_to="")
        model = GA.BaseGA(epoch=50, pop_size=20, pc=0.8, pm=0.15, selection="tournament", k_way=k_way, cross_over="one_point")
        model.solve(problem=problem_jssp)

        total_score += model.g_best.target.fitness

    print(f"Total score: {total_score}")
    return total_score

In [98]:
# Define the search space for the parameters
space = [
    Real(0.8, 0.99, name='pc'),
    Real(0.1, 0.99, name='pm'),
    Real(0.1, 0.99, name='k_way')
]

# Perform Bayesian optimization
res = gp_minimize(compute_overall_score_GA, space, n_calls=20, random_state=0)

Params: [0.9126404774627536, 0.8513965162371057, 0.8635715996842537]
Total score: 2395782.0
Params: [0.9609778303689839, 0.6549716901395155, 0.442099719490503]
Total score: 2401465.0
Params: [0.8565315752434498, 0.15047454981252445, 0.34266410217630083]
Total score: 2405114.0
Params: [0.8907563722910565, 0.8228301686101891, 0.5271796834138012]
Total score: 2400269.0
Params: [0.8746291112591577, 0.8441100995482662, 0.40028258277136897]
Total score: 2405356.0
Params: [0.9231526556897275, 0.4277349704580877, 0.9518680914682115]
Total score: 2395462.0
Params: [0.8266666482784026, 0.8743776599390085, 0.5215111602936024]
Total score: 2402688.0
Params: [0.9521730428761325, 0.5632249568005724, 0.7042027818058747]
Total score: 2398084.0
Params: [0.9369202043979242, 0.6179976149468455, 0.5782621742096196]
Total score: 2401616.0
Params: [0.944136968621248, 0.19425777039713504, 0.5215043732185252]
Total score: 2401536.0
Params: [0.99, 0.99, 0.99]
Total score: 2396338.0
Params: [0.8455952661360099,

In [100]:
# Output the best parameters found
print("Best parameters found:")
print(f"pc: {res.x[0]}")
print(f"pm: {res.x[1]}")
print(f"k_way: {res.x[2]}")

print("Best score achieved:")
print(res.fun)

Best parameters found:
pc: 0.99
pm: 0.99
k_way: 0.9278618687340392
Best score achieved:
2394425.0


In [121]:
def compute_overall_score_ACO(params):
    sample_count, intent_factor, zeta = params
    print(f"Params: {params}")
    total_score = 0

    for filename, content in file_data.items():
        data = read_file_input(f"{folder_path}/{filename}")
        problem_jssp = JSSP(bounds=PermutationVar(valid_set=[i for i in range(data.shape[0])], name="per"), processing_times=data, minmax="min", log_to="")
        model = ACOR.OriginalACOR(epoch=50, pop_size=20, sample_count=int(sample_count), intent_factor=intent_factor, zeta=zeta)
        model.solve(problem=problem_jssp)
        
        total_score += model.g_best.target.fitness

    print(f"Total score: {total_score}")
    return total_score

In [122]:
# Define the search space for the parameters
space = [
    Integer(10, 25, name='sample_count'),
    Real(0.1, 0.99, name='intent_factor'),
    Real(0.1, 4.99, name='zeta')
]

# Perform Bayesian optimization
res = gp_minimize(compute_overall_score_ACO, space, n_calls=20, random_state=0)

Params: [19, 0.8513965162371057, 4.295354070175282]
Total score: 2420626.0
Params: [23, 0.6549716901395155, 1.979626548661303]
Total score: 2413973.0
Params: [14, 0.15047454981252445, 1.433289280496754]
Total score: 2419249.0
Params: [17, 0.8228301686101891, 2.447088372914031]
Total score: 2422630.0
Params: [16, 0.8441100995482662, 1.749867224440443]
Total score: 2421428.0
Params: [20, 0.4277349704580877, 4.780488727280398]
Total score: 2418702.0
Params: [12, 0.8743776599390085, 2.415943341388445]
Total score: 2427355.0
Params: [22, 0.5632249568005724, 3.4197209022817168]
Total score: 2418409.0
Params: [21, 0.6179976149468455, 2.7277550920056632]
Total score: 2416828.0
Params: [21, 0.19425777039713504, 2.4159060506051557]
Total score: 2414723.0
Params: [25, 0.1, 0.1]
Total score: 2395692.0


/home/marin/.local/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [25, 0.1, 0.1] before, using random point [14, 0.5112653846707935, 3.2471890742741687]
  warnings.warn(


Params: [14, 0.5112653846707935, 3.2471890742741687]
Total score: 2427715.0
Params: [22, 0.1, 0.1]
Total score: 2399203.0


/home/marin/.local/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [25, 0.1, 0.1] before, using random point [24, 0.7228585525869469, 1.9553028793454266]
  warnings.warn(


Params: [24, 0.7228585525869469, 1.9553028793454266]
Total score: 2412032.0


...

Params: [14, 0.10503393637640371, 3.712718096571511]
Total score: 2425670.0


/home/marin/.local/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [25, 0.1, 0.1] before, using random point [24, 0.783306190731854, 1.2813587697258368]
  warnings.warn(


Params: [24, 0.783306190731854, 1.2813587697258368]
Total score: 2411056.0


/home/marin/.local/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [25, 0.1, 0.1] before, using random point [24, 0.40445532338189216, 1.3800140340196885]
  warnings.warn(


Params: [24, 0.40445532338189216, 1.3800140340196885]
Total score: 2413422.0
Params: [25, 0.99, 0.1]
Total score: 2398448.0


In [123]:
# Output the best parameters found
print("Best parameters found:")
print(f"sample_count: {res.x[0]}")
print(f"intent_factor: {res.x[1]}")
print(f"zeta: {res.x[2]}")

print("Best score achieved:")
print(res.fun)

Best parameters found:
sample_count: 25
intent_factor: 0.1
zeta: 0.1
Best score achieved:
2395692.0
